In [4]:
library(xlsx)
ACC=read.xlsx("../Data/ACC.xlsx", sheetIndex = 1)

In [5]:
head(ACC)

Date,Spot_Ret,Futures_Ret
20120103,0.004249639,0.0060496779
20120104,-0.035056971,-0.0367694978
20120105,0.007532907,0.0069533510
20120106,-0.008620159,-0.0057836024
20120107,-0.001179192,-0.0053198809
20120109,-0.001953259,-0.0001808318


In [6]:
##Multivariate Time-series modelling
#VAR and Granger causality test
###############################################


##VAR is an extension of AR models, hence, all variables must be stationary
##Popularised by Sims (1980)
#A VAR is a systems regression model (more than one dependent variable)
#Simplest case is a Bivariate-VAR
#Can be extentended to n variables

##Select file (Import ACC file)
##Check stationarity
#Do ADF test for both returns
require(urca)
attach(ACC)

Loading required package: urca


In [7]:
x_adf_prices=ur.df(Spot_Ret, type = "drift")
x_adf_prices@teststat[1] #will give required output as Eviews 0.3107101
x_adf_prices@cval [1,2] #Select 5% value tau2 values here -2.87
#Can reject null #Hence, series is stationary
#Reject when Test statistic is less than -2.87 (ADF is a left-tail test)

[1] -18.39532

[1] -2.86

In [8]:
##Similarly for Futures_ret
##############################
y_adf_prices=ur.df(Futures_Ret, type = "drift")
y_adf_prices@teststat[1] #will give required output as Eviews 0.3107101
y_adf_prices@cval [1,2] #Select 5% value tau2 values here -2.87
#Can reject null #Hence, series is stationary

[1] -18.28384

[1] -2.86

In [9]:
#Both series are stationary, hence we can apply VAR
######################################################


##Choosing the right lag for the model, using Information criteria
#Install "vars" package
library(vars)
y2=VARselect(ACC[,2:3], lag.max = 10, type = "const")#Gives lag selection criterias for a VAR with constant only
y2

Loading required package: MASS
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich
Loading required package: lmtest


,1,2,3,4,5,6,7,8,9,10
AIC(n),-1.981979e+01,-1.981845e+01,-1.981796e+01,-1.982260e+01,-1.981427e+01,-1.980980e+01,-1.981844e+01,-1.982003e+01,-1.981012e+01,-1.980164e+01
HQ(n),-1.980529e+01,-1.979429e+01,-1.978413e+01,-1.977911e+01,-1.976110e+01,-1.974697e+01,-1.974595e+01,-1.973787e+01,-1.971829e+01,-1.970015e+01
SC(n),-1.978220e+01,-1.975580e+01,-1.973025e+01,-1.970983e+01,-1.967644e+01,-1.964691e+01,-1.963049e+01,-1.960702e+01,-1.957205e+01,-1.953851e+01
FPE(n),2.468159e-09,2.471474e-09,2.472695e-09,2.461241e-09,2.481850e-09,2.492967e-09,2.471524e-09,2.467609e-09,2.492217e-09,2.513457e-09


In [10]:
#SIC lag selection
y2$selection[3]#lags as per SC (Schwartz Criterion)

SC(n) 
    1

In [11]:
#We will run VAR with lag 1
####################################################
#Run the VAR model with appropriate lag length
var_model=VAR(ACC[,2:3],p=1, type= "const")
var_model
summary(var_model)#All details
########################################


VAR Estimation Results:

Estimated coefficients for equation Spot_Ret: 
Call:
Spot_Ret = Spot_Ret.l1 + Futures_Ret.l1 + const 

   Spot_Ret.l1 Futures_Ret.l1          const 
 -0.2232168302   0.2924321603   0.0002511874 


Estimated coefficients for equation Futures_Ret: 
Call:
Futures_Ret = Spot_Ret.l1 + Futures_Ret.l1 + const 

   Spot_Ret.l1 Futures_Ret.l1          const 
  0.0556873438  -0.0057986813   0.0002607756 




VAR Estimation Results:
Endogenous variables: Spot_Ret, Futures_Ret 
Deterministic variables: const 
Sample size: 743 
Log Likelihood: 5263.079 
Roots of the characteristic polynomial:
0.2821 0.05313
Call:
VAR(y = ACC[, 2:3], p = 1, type = "const")


Estimation results for equation Spot_Ret: 
Spot_Ret = Spot_Ret.l1 + Futures_Ret.l1 + const 

                 Estimate Std. Error t value Pr(>|t|)
Spot_Ret.l1    -0.2232168  0.1859274  -1.201    0.230
Futures_Ret.l1  0.2924322  0.1887947   1.549    0.122
const           0.0002512  0.0005970   0.421    0.674


Residual standard error: 0.01627 on 740 degrees of freedom
Multiple R-Squared: 0.006717,	Adjusted R-squared: 0.004032 
F-statistic: 2.502 on 2 and 740 DF,  p-value: 0.08261 


Estimation results for equation Futures_Ret: 
Futures_Ret = Spot_Ret.l1 + Futures_Ret.l1 + const 

                 Estimate Std. Error t value Pr(>|t|)
Spot_Ret.l1     0.0556873  0.1834744   0.304    0.762
Futures_Ret.l1 -0.0057987  0.1863039  -0.031    0.975


In [12]:
#Run appropriate model directly with SIC lags
var_model=VAR(ACC[,2:3], type= "const", lag.max = 10, ic="SC")
var_model
summary(var_model)#All details
###############################################
##Block significance and causality tests
#Granger's causality test can be used to test the direction of causal relationship 
#We can use the results of VAR model in var object "var_model"


VAR Estimation Results:

Estimated coefficients for equation Spot_Ret: 
Call:
Spot_Ret = Spot_Ret.l1 + Futures_Ret.l1 + const 

   Spot_Ret.l1 Futures_Ret.l1          const 
 -0.2232168302   0.2924321603   0.0002511874 


Estimated coefficients for equation Futures_Ret: 
Call:
Futures_Ret = Spot_Ret.l1 + Futures_Ret.l1 + const 

   Spot_Ret.l1 Futures_Ret.l1          const 
  0.0556873438  -0.0057986813   0.0002607756 




VAR Estimation Results:
Endogenous variables: Spot_Ret, Futures_Ret 
Deterministic variables: const 
Sample size: 743 
Log Likelihood: 5263.079 
Roots of the characteristic polynomial:
0.2821 0.05313
Call:
VAR(y = ACC[, 2:3], type = "const", lag.max = 10, ic = "SC")


Estimation results for equation Spot_Ret: 
Spot_Ret = Spot_Ret.l1 + Futures_Ret.l1 + const 

                 Estimate Std. Error t value Pr(>|t|)
Spot_Ret.l1    -0.2232168  0.1859274  -1.201    0.230
Futures_Ret.l1  0.2924322  0.1887947   1.549    0.122
const           0.0002512  0.0005970   0.421    0.674


Residual standard error: 0.01627 on 740 degrees of freedom
Multiple R-Squared: 0.006717,	Adjusted R-squared: 0.004032 
F-statistic: 2.502 on 2 and 740 DF,  p-value: 0.08261 


Estimation results for equation Futures_Ret: 
Futures_Ret = Spot_Ret.l1 + Futures_Ret.l1 + const 

                 Estimate Std. Error t value Pr(>|t|)
Spot_Ret.l1     0.0556873  0.1834744   0.304    0.762
Futures_Ret.l1 -0.0057987  0.1863039

In [13]:
##Testing for granger causality
#Null Hypothesis: spot_ret does not Granger cause futures_ret
causality(var_model, cause = "Spot_Ret")$Granger


	Granger causality H0: Spot_Ret do not Granger-cause Futures_Ret

data:  VAR object var_model
F-Test = 0.092122, df1 = 1, df2 = 1480, p-value = 0.7615


In [14]:
#Infer the results
#Since p-value is not less than 0.05, we can not reject the null
#And, conclude that spot_ret does not Granger cause futures_ret
###################################################

##Similarly, conduct Granger test for reverse causality
#Null Hypothesis: futures_ret does not Granger cause spot_ret
causality(var_model, cause = "Futures_Ret")$Granger


	Granger causality H0: Futures_Ret do not Granger-cause Spot_Ret

data:  VAR object var_model
F-Test = 2.3992, df1 = 1, df2 = 1480, p-value = 0.1216


In [15]:
#Infer the results
#Since p-value is not less than 0.05, we can not reject the null
#And, conclude that futures_ret does not Granger cause spot_ret

##Overall, there is no causality in these two variables
##Both markets are efficient and no arbitrage opprtunity exists.
################################################################